In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
#loading all data
train = pd.read_csv('../input/wns2019/train.csv', delimiter=',')
train.dataframeName = 'train.csv'
train['impression_time']= pd.to_datetime(train['impression_time']) 
train.head(5)

item = pd.read_csv('../input/wns2019/item_data.csv', delimiter=',')
item.dataframeName = 'item_data.csv'
item.head(5)

view = pd.read_csv('../input/wns2019/view_log.csv', delimiter=',')
view.dataframeName = 'view_log.csv'
view['server_time']= pd.to_datetime(view['server_time']) 
view.head(5)

test = pd.read_csv('../input/wns2019/test.csv', delimiter=',')
test.dataframeName = 'test.csv'
test['impression_time']= pd.to_datetime(test['impression_time']) 
test.head(5)

sample_submit = pd.read_csv('../input/wns2019/sample_submission.csv', delimiter=',')
sample_submit.dataframeName = 'sample_submission.csv'
sample_submit.head(5)

,impression_id,is_click
0,a9e7126a585a69a32bc7414e9d0c0ada,0
1,caac14a5bf2ba283db7708bb34855760,0
2,13f10ba306a19ce7bec2f3cae507b698,0
3,39c4b4dc0e9701b55a0a4f072008fb3f,0
4,bf5a572cca75f5fc67f4b14e58b11d70,0


In [3]:
train.head(5)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [4]:
#picking relevent columns from itemdata
dummy_cat1=pd.get_dummies(item['category_1'],prefix = 'cat1')
counts = item.category_2.value_counts()
cat1_cols = dummy_cat1.columns.tolist()
cat1_cols.append('item_id')

repl = counts[counts <= 4000 ].index
dummy_cat2 = pd.get_dummies(item.category_2.replace(repl, 'other'),prefix = 'cat2')
cat2_cols = dummy_cat2.columns.tolist()
cat2_cols.append('item_id')

counts = item.category_3.value_counts()
repl = counts[counts <= 1800 ].index
dummy_cat3 = pd.get_dummies(item.category_3.replace(repl, 'other'),prefix = 'cat3')
cat3_cols = dummy_cat3.columns.tolist()
cat3_cols.append('item_id')

counts = item.product_type.value_counts()
repl = counts[counts <= 632 ].index
dummy_ptype = pd.get_dummies(item.product_type.replace(repl, 'other'),prefix = 'ptype')
ptype_cols = dummy_ptype.columns.tolist()
ptype_cols.append('item_id')

binned_price = pd.qcut(item['item_price'], q=20,labels = False)
dummy_price = pd.get_dummies(binned_price ,prefix = 'price')
price_cols = dummy_price.columns.tolist()
price_cols.append('item_id')

itm = pd.concat([item[['item_id','item_price']],dummy_cat1,dummy_cat2,dummy_cat3,dummy_ptype,dummy_price],axis=1)

In [5]:
list1=itm.columns.tolist()
list1.pop(0)
list1.pop(0)
list1

['cat1_0',
 'cat1_1',
 'cat1_2',
 'cat1_4',
 'cat1_5',
 'cat1_6',
 'cat1_7',
 'cat1_8',
 'cat1_9',
 'cat1_10',
 'cat1_11',
 'cat1_12',
 'cat1_13',
 'cat1_14',
 'cat1_15',
 'cat1_16',
 'cat1_17',
 'cat2_3',
 'cat2_8',
 'cat2_9',
 'cat2_38',
 'cat2_39',
 'cat2_42',
 'cat2_44',
 'cat2_51',
 'cat2_57',
 'cat2_74',
 'cat2_other',
 'cat3_36',
 'cat3_62',
 'cat3_73',
 'cat3_84',
 'cat3_113',
 'cat3_114',
 'cat3_151',
 'cat3_157',
 'cat3_159',
 'cat3_162',
 'cat3_223',
 'cat3_244',
 'cat3_252',
 'cat3_279',
 'cat3_292',
 'cat3_other',
 'ptype_2106',
 'ptype_2874',
 'ptype_5036',
 'ptype_5164',
 'ptype_5609',
 'ptype_5622',
 'ptype_7455',
 'ptype_7791',
 'ptype_10412',
 'ptype_other',
 'price_0',
 'price_1',
 'price_2',
 'price_3',
 'price_4',
 'price_5',
 'price_6',
 'price_7',
 'price_8',
 'price_9',
 'price_10',
 'price_11',
 'price_12',
 'price_13',
 'price_14',
 'price_15',
 'price_16',
 'price_17',
 'price_18',
 'price_19']

In [6]:
(cat1_cols)

['cat1_0',
 'cat1_1',
 'cat1_2',
 'cat1_4',
 'cat1_5',
 'cat1_6',
 'cat1_7',
 'cat1_8',
 'cat1_9',
 'cat1_10',
 'cat1_11',
 'cat1_12',
 'cat1_13',
 'cat1_14',
 'cat1_15',
 'cat1_16',
 'cat1_17',
 'item_id']

In [7]:
#merging view log and itm
df = pd.merge(view[['user_id','session_id','item_id','server_time','device_type']], itm, on='item_id', how='left')

In [8]:
df.head(10)

,user_id,session_id,item_id,server_time,device_type,item_price,cat1_0,cat1_1,cat1_2,cat1_4,...,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19
0,4557,112333,32970,2018-10-15 08:58:00,android,54685.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,74788,503590,7640,2018-10-15 08:58:00,android,1376.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23628,573960,128855,2018-10-15 08:58:00,android,4544.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2430,121691,12774,2018-10-15 08:58:00,android,904.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19227,218564,28296,2018-10-15 08:58:00,android,2304.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,87387,635314,86660,2018-10-15 08:58:00,android,10304.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,348,482459,35370,2018-10-15 08:58:00,android,480.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,19227,218564,100065,2018-10-15 08:58:00,android,1587.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,30183,947548,86558,2018-10-15 08:58:00,android,9920.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,31754,941129,127900,2018-10-15 08:58:00,android,3520.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#merging train and view
big_merge_key = pd.merge(view[['user_id','session_id','server_time','item_id']], train, on = 'user_id', how = 'left')
big_merge_key.head(10)

,user_id,session_id,server_time,item_id,impression_id,impression_time,app_code,os_version,is_4G,is_click
0,4557,112333,2018-10-15 08:58:00,32970,6dd9544216149618da55e183f73e2b56,2018-11-26 14:37:00,213.0,latest,1.0,0.0
1,4557,112333,2018-10-15 08:58:00,32970,6ba86f2d1de0989ffbf73cc37f5825c9,2018-11-26 14:35:00,213.0,latest,1.0,0.0
2,74788,503590,2018-10-15 08:58:00,7640,d3a1d384972d1253ef47b4bdc0c53320,2018-11-27 11:09:00,207.0,intermediate,0.0,0.0
3,74788,503590,2018-10-15 08:58:00,7640,58825cb05cf2694bac1da9318ef90091,2018-11-27 15:59:00,207.0,intermediate,0.0,0.0
4,23628,573960,2018-10-15 08:58:00,128855,93367ed3fc274b77debadd3de60b954c,2018-12-08 17:42:00,386.0,old,0.0,0.0
5,23628,573960,2018-10-15 08:58:00,128855,81fd65e01c10c3b59fdf735e90104265,2018-12-10 12:52:00,386.0,old,0.0,0.0
6,2430,121691,2018-10-15 08:58:00,12774,1c5589bf644147425c5121c89f2d6d34,2018-11-20 12:18:00,434.0,old,0.0,0.0
7,2430,121691,2018-10-15 08:58:00,12774,7552663f68b5eba5b1e88899419a4a01,2018-11-17 14:11:00,434.0,old,0.0,0.0
8,2430,121691,2018-10-15 08:58:00,12774,989f78982634dc53561c1c70e229d226,2018-11-17 14:19:00,434.0,old,0.0,1.0
9,2430,121691,2018-10-15 08:58:00,12774,917cec83af22cd3d38a08fcc1b26dacd,2018-11-20 12:57:00,434.0,old,0.0,0.0


In [10]:
big_merge_key.describe(include='all')

,user_id,session_id,server_time,item_id,impression_id,impression_time,app_code,os_version,is_4G,is_click
count,1.148687e+07,1.148687e+07,11486867,1.148687e+07,11052882,11052882,1.105288e+07,11052882,1.105288e+07,1.105288e+07
unique,NaN,NaN,82840,NaN,237609,36461,NaN,3,NaN,NaN
top,NaN,NaN,2018-11-14 23:27:00,NaN,01c8a64a2b3c66c05c2dbf9df27510eb,2018-12-02 00:01:00,NaN,latest,NaN,NaN
freq,NaN,NaN,709,NaN,2906,3311,NaN,5783941,NaN,NaN
first,NaN,NaN,2018-10-15 08:58:00,NaN,NaN,2018-11-15 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,2018-12-11 23:59:00,NaN,NaN,2018-12-13 07:43:00,NaN,NaN,NaN,NaN
mean,4.637694e+04,5.457425e+05,NaN,6.615014e+04,NaN,NaN,2.511502e+02,NaN,3.086756e-01,5.985959e-02
std,2.690204e+04,3.140060e+05,NaN,3.827350e+04,NaN,NaN,1.337467e+02,NaN,4.619470e-01,2.372265e-01
min,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00
25%,2.319700e+04,2.749970e+05,NaN,3.323700e+04,NaN,NaN,1.630000e+02,NaN,0.000000e+00,0.000000e+00


In [11]:
#time difference calculation
big_merge_key['time_diff'] = big_merge_key['impression_time'] - big_merge_key['server_time']
big_merge_key['time_diff_s'] = big_merge_key['time_diff']/np.timedelta64(1,'s')
big_merge_key['time_diff_h'] = big_merge_key['time_diff']/np.timedelta64(1,'h')
big_merge_key['time_diff_d'] = big_merge_key['time_diff']/np.timedelta64(1,'D')
big_merge_key.head(10)

,user_id,session_id,server_time,item_id,impression_id,impression_time,app_code,os_version,is_4G,is_click,time_diff,time_diff_s,time_diff_h,time_diff_d
0,4557,112333,2018-10-15 08:58:00,32970,6dd9544216149618da55e183f73e2b56,2018-11-26 14:37:00,213.0,latest,1.0,0.0,42 days 05:39:00,3649140.0,1013.650000,42.235417
1,4557,112333,2018-10-15 08:58:00,32970,6ba86f2d1de0989ffbf73cc37f5825c9,2018-11-26 14:35:00,213.0,latest,1.0,0.0,42 days 05:37:00,3649020.0,1013.616667,42.234028
2,74788,503590,2018-10-15 08:58:00,7640,d3a1d384972d1253ef47b4bdc0c53320,2018-11-27 11:09:00,207.0,intermediate,0.0,0.0,43 days 02:11:00,3723060.0,1034.183333,43.090972
3,74788,503590,2018-10-15 08:58:00,7640,58825cb05cf2694bac1da9318ef90091,2018-11-27 15:59:00,207.0,intermediate,0.0,0.0,43 days 07:01:00,3740460.0,1039.016667,43.292361
4,23628,573960,2018-10-15 08:58:00,128855,93367ed3fc274b77debadd3de60b954c,2018-12-08 17:42:00,386.0,old,0.0,0.0,54 days 08:44:00,4697040.0,1304.733333,54.363889
5,23628,573960,2018-10-15 08:58:00,128855,81fd65e01c10c3b59fdf735e90104265,2018-12-10 12:52:00,386.0,old,0.0,0.0,56 days 03:54:00,4852440.0,1347.900000,56.162500
6,2430,121691,2018-10-15 08:58:00,12774,1c5589bf644147425c5121c89f2d6d34,2018-11-20 12:18:00,434.0,old,0.0,0.0,36 days 03:20:00,3122400.0,867.333333,36.138889
7,2430,121691,2018-10-15 08:58:00,12774,7552663f68b5eba5b1e88899419a4a01,2018-11-17 14:11:00,434.0,old,0.0,0.0,33 days 05:13:00,2869980.0,797.216667,33.217361
8,2430,121691,2018-10-15 08:58:00,12774,989f78982634dc53561c1c70e229d226,2018-11-17 14:19:00,434.0,old,0.0,1.0,33 days 05:21:00,2870460.0,797.350000,33.222917
9,2430,121691,2018-10-15 08:58:00,12774,917cec83af22cd3d38a08fcc1b26dacd,2018-11-20 12:57:00,434.0,old,0.0,0.0,36 days 03:59:00,3124740.0,867.983333,36.165972


In [12]:
big_merge_key2 = big_merge_key[big_merge_key.time_diff_s >= 0]

In [13]:
big_merge_key2.describe(include='all')

,user_id,session_id,server_time,item_id,impression_id,impression_time,app_code,os_version,is_4G,is_click,time_diff,time_diff_s,time_diff_h,time_diff_d
count,8.580839e+06,8.580839e+06,8580839,8.580839e+06,8580839,8580839,8.580839e+06,8580839,8.580839e+06,8.580839e+06,8580839,8.580839e+06,8.580839e+06,8.580839e+06
unique,NaN,NaN,82648,NaN,236022,36433,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,2018-11-14 23:27:00,NaN,01c8a64a2b3c66c05c2dbf9df27510eb,2018-12-02 00:01:00,NaN,latest,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,699,NaN,1741,2583,NaN,4474950,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,2018-10-15 08:58:00,NaN,NaN,2018-11-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,2018-12-11 23:59:00,NaN,NaN,2018-12-13 07:43:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,4.657328e+04,5.456849e+05,NaN,6.612563e+04,NaN,NaN,2.511379e+02,NaN,3.082119e-01,5.842785e-02,21 days 11:25:40.798003,1.855541e+06,5.154280e+02,2.147617e+01
std,2.691867e+04,3.139551e+05,NaN,3.829867e+04,NaN,NaN,1.335379e+02,NaN,4.617547e-01,2.345507e-01,14 days 03:41:43.610381,1.222904e+06,3.396954e+02,1.415398e+01
min,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0 days 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.342900e+04,2.750320e+05,NaN,3.319000e+04,NaN,NaN,1.630000e+02,NaN,0.000000e+00,0.000000e+00,9 days 03:47:00,7.912200e+05,2.197833e+02,9.157639e+00


In [14]:
big_merge_cat1 = pd.merge(big_merge_key2, itm[cat1_cols] , on = 'item_id', how = 'left')
cat1_cols.remove('item_id')
big_merge_cat1 = big_merge_cat1.groupby(['impression_id','user_id'],as_index = False)[cat1_cols].sum()
big_merge_cat1.head(10)

,impression_id,user_id,cat1_0,cat1_1,cat1_2,cat1_4,cat1_5,cat1_6,cat1_7,cat1_8,cat1_9,cat1_10,cat1_11,cat1_12,cat1_13,cat1_14,cat1_15,cat1_16,cat1_17
0,00003e3b9e5336685200ae85d21b4f5e,38051,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,000053b1e684c9e7ea73727b2238ce18,65897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
2,00005d011db80a956aab176cc94d1d37,4842,1.0,2.0,0.0,4.0,0.0,0.0,2.0,1.0,12.0,4.0,1.0,7.0,1.0,3.0,0.0,1.0,3.0
3,0000b2815cc3c2b56867cbbf4d36efa5,16990,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,8.0,0.0,1.0,1.0,0.0,0.0,2.0
4,0001261e2060303a06ba6c64d676d639,8726,5.0,12.0,0.0,4.0,0.0,0.0,5.0,2.0,17.0,4.0,2.0,7.0,4.0,1.0,1.0,0.0,2.0
5,000133296ef6b63b0210f224e1347365,928,32.0,40.0,0.0,33.0,0.0,0.0,16.0,8.0,96.0,10.0,27.0,21.0,41.0,25.0,20.0,51.0,70.0
6,00017961865c4f766fdbb3cd8fe0bfb0,4431,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
7,00019d812c1173c8a69c656a40fa8767,24014,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,1.0,0.0,3.0
8,00029153d12ae1c9abe59c17ff2e0895,32875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,000298d8a61b4f7b0fe5c7fd3d8c4c9c,32381,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [15]:
big_merge_cat1.describe(include = 'all')

,impression_id,user_id,cat1_0,cat1_1,cat1_2,cat1_4,cat1_5,cat1_6,cat1_7,cat1_8,cat1_9,cat1_10,cat1_11,cat1_12,cat1_13,cat1_14,cat1_15,cat1_16,cat1_17
count,236022,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000
unique,236022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,408a9c4a79800232ac656249af3162eb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,46452.651880,1.390777,4.276856,0.000008,3.392400,0.000004,0.544305,2.054635,1.915097,4.796820,2.097982,2.786452,1.608943,2.836892,1.888150,0.812331,1.457809,4.473439
std,NaN,26802.157679,2.909922,7.348156,0.002911,6.423035,0.002058,1.521456,4.225658,3.882994,8.369551,4.117131,5.182652,3.544811,5.342905,3.812189,2.069637,3.474131,7.800068
min,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,23197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,46584.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,2.000000
75%,NaN,69691.500000,2.000000,5.000000,0.000000,4.000000,0.000000,0.000000,2.000000,2.000000,6.000000,2.000000,3.000000,2.000000,3.000000,2.000000,1.000000,2.000000,5.000000


In [16]:
big_merge_cat2 = pd.merge(big_merge_key2, itm[cat2_cols] , on = 'item_id', how = 'left')
cat2_cols.remove('item_id')
big_merge_cat2 = big_merge_cat2.groupby(['impression_id','user_id'],as_index = False)[cat2_cols].sum()
big_merge_cat2.head(10)

,impression_id,user_id,cat2_3,cat2_8,cat2_9,cat2_38,cat2_39,cat2_42,cat2_44,cat2_51,cat2_57,cat2_74,cat2_other
0,00003e3b9e5336685200ae85d21b4f5e,38051,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,6.0
1,000053b1e684c9e7ea73727b2238ce18,65897,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
2,00005d011db80a956aab176cc94d1d37,4842,0.0,2.0,10.0,0.0,1.0,1.0,1.0,0.0,7.0,3.0,17.0
3,0000b2815cc3c2b56867cbbf4d36efa5,16990,3.0,2.0,1.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,11.0
4,0001261e2060303a06ba6c64d676d639,8726,4.0,2.0,6.0,2.0,0.0,8.0,7.0,1.0,7.0,1.0,28.0
5,000133296ef6b63b0210f224e1347365,928,41.0,20.0,30.0,19.0,39.0,8.0,16.0,21.0,16.0,7.0,273.0
6,00017961865c4f766fdbb3cd8fe0bfb0,4431,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0
7,00019d812c1173c8a69c656a40fa8767,24014,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0
8,00029153d12ae1c9abe59c17ff2e0895,32875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
9,000298d8a61b4f7b0fe5c7fd3d8c4c9c,32381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0


In [17]:
big_merge_cat3 = pd.merge(big_merge_key2, itm[cat3_cols] , on = 'item_id', how = 'left')
cat3_cols.remove('item_id')
big_merge_cat3 = big_merge_cat3.groupby(['impression_id','user_id'],as_index = False)[cat3_cols].sum()
big_merge_cat3.head(10)

,impression_id,user_id,cat3_36,cat3_62,cat3_73,cat3_84,cat3_113,cat3_114,cat3_151,cat3_157,cat3_159,cat3_162,cat3_223,cat3_244,cat3_252,cat3_279,cat3_292,cat3_other
0,00003e3b9e5336685200ae85d21b4f5e,38051,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1,000053b1e684c9e7ea73727b2238ce18,65897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
2,00005d011db80a956aab176cc94d1d37,4842,0.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,32.0
3,0000b2815cc3c2b56867cbbf4d36efa5,16990,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0
4,0001261e2060303a06ba6c64d676d639,8726,2.0,2.0,3.0,0.0,1.0,1.0,0.0,6.0,1.0,0.0,0.0,6.0,0.0,0.0,1.0,43.0
5,000133296ef6b63b0210f224e1347365,928,6.0,19.0,21.0,1.0,17.0,13.0,3.0,5.0,0.0,5.0,3.0,25.0,0.0,1.0,7.0,364.0
6,00017961865c4f766fdbb3cd8fe0bfb0,4431,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0
7,00019d812c1173c8a69c656a40fa8767,24014,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,9.0
8,00029153d12ae1c9abe59c17ff2e0895,32875,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,000298d8a61b4f7b0fe5c7fd3d8c4c9c,32381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0


In [18]:
big_merge_ptype = pd.merge(big_merge_key2, itm[ptype_cols] , on = 'item_id', how = 'left')
ptype_cols.remove('item_id')
big_merge_ptype = big_merge_ptype.groupby(['impression_id','user_id'],as_index = False)[ptype_cols].sum()
big_merge_ptype.head(10)

,impression_id,user_id,ptype_2106,ptype_2874,ptype_5036,ptype_5164,ptype_5609,ptype_5622,ptype_7455,ptype_7791,ptype_10412,ptype_other
0,00003e3b9e5336685200ae85d21b4f5e,38051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
1,000053b1e684c9e7ea73727b2238ce18,65897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,00005d011db80a956aab176cc94d1d37,4842,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,41.0
3,0000b2815cc3c2b56867cbbf4d36efa5,16990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,23.0
4,0001261e2060303a06ba6c64d676d639,8726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0
5,000133296ef6b63b0210f224e1347365,928,3.0,16.0,0.0,8.0,0.0,1.0,0.0,3.0,0.0,459.0
6,00017961865c4f766fdbb3cd8fe0bfb0,4431,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0
7,00019d812c1173c8a69c656a40fa8767,24014,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
8,00029153d12ae1c9abe59c17ff2e0895,32875,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,000298d8a61b4f7b0fe5c7fd3d8c4c9c,32381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [19]:
big_merge_price = pd.merge(big_merge_key2, itm[price_cols] , on = 'item_id', how = 'left')
price_cols.remove('item_id')
big_merge_price = big_merge_price.groupby(['impression_id','user_id'],as_index = False)[price_cols].sum()
big_merge_price.head(10)

,impression_id,user_id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,...,price_10,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19
0,00003e3b9e5336685200ae85d21b4f5e,38051,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000053b1e684c9e7ea73727b2238ce18,65897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,00005d011db80a956aab176cc94d1d37,4842,8.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0,...,1.0,9.0,2.0,3.0,1.0,2.0,0.0,0.0,1.0,1.0
3,0000b2815cc3c2b56867cbbf4d36efa5,16990,0.0,0.0,3.0,0.0,0.0,0.0,1.0,7.0,...,0.0,1.0,0.0,1.0,1.0,3.0,0.0,3.0,3.0,0.0
4,0001261e2060303a06ba6c64d676d639,8726,1.0,6.0,1.0,4.0,2.0,0.0,3.0,3.0,...,6.0,4.0,0.0,11.0,0.0,2.0,1.0,3.0,6.0,3.0
5,000133296ef6b63b0210f224e1347365,928,39.0,39.0,11.0,8.0,29.0,11.0,31.0,17.0,...,7.0,13.0,10.0,84.0,12.0,13.0,57.0,3.0,7.0,64.0
6,00017961865c4f766fdbb3cd8fe0bfb0,4431,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
7,00019d812c1173c8a69c656a40fa8767,24014,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0
8,00029153d12ae1c9abe59c17ff2e0895,32875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,000298d8a61b4f7b0fe5c7fd3d8c4c9c,32381,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [20]:
session_count = big_merge_key2.groupby(['user_id','impression_id'],as_index = False).agg({ 'session_id' : ['nunique']})
session_count.columns = ['user_id','impression_id','session_count']
session_count.head(10)

,user_id,impression_id,session_count
0,0,5f98c2c31a8006e510448c02ec74d50f,7
1,2,010ed37e44e2fdc175b4c5c6c930805a,30
2,2,2b12c0d47f5821a5adb3bfd973d0f708,28
3,2,577c63f9937fa0e8d4650ddf1510a03f,30
4,2,f85e4bf0e34f3ee66add56229845e4db,30
5,3,75f1a891e432ee31eeeb7ae145c665f1,1
6,5,de0f7e146d594e3868c1f4aae5600c2f,1
7,5,fb4cd2d75ba5915913d52e43c780c3d2,1
8,6,f178a1b8e28622bddf49657470a15cd3,4
9,8,b69052087938225e152c4942e5e81b1b,31


In [21]:
# session_nbr_diff_item = big_merge2.groupby(['impression_id','user_id','session_id'],as_index = False).agg({'item_id' : ['nunique']})
# session_nbr_diff_item.columns = ['impression_id','user_id','session_id','count_item']
# user_avg_diff_item = session_nbr_diff_item.groupby(['impression_id','user_id'],as_index = False).agg({'count_item' : ['mean']})
# user_avg_diff_item.columns = ['impression_id','user_id','count_item_mean']
# user_avg_diff_item.head(10)

In [22]:
# session_stamp_min_max = big_merge2.groupby(['impression_id','user_id','session_id'],as_index = False).agg({'server_time' : [min,max]})
# session_stamp_min_max.columns = ['impression_id','user_id','session_id','min_stamp','max_stamp']

In [23]:
# session_price_min_max = big_merge2.groupby(['impression_id','user_id','session_id'],as_index = False).agg({'item_price' : [min,max,'mean']})
# session_price_min_max.columns = ['impression_id','user_id','session_id','price_min','price_max','price_mean']

In [24]:
# user_price_min_max = session_price_min_max.groupby(['impression_id','user_id'],as_index = False).agg({'price_min' : [min], 'price_max' : [max], 'price_mean' : ['mean']})
# user_price_min_max.columns = ['impression_id','user_id','price_min','price_max','price_mean']
# user_price_min_max.head(10)

In [25]:
# merging all variables
big_merge = pd.merge(train, big_merge_cat1 , on = ['impression_id','user_id'], how = 'left')
big_merge = pd.merge(big_merge, big_merge_cat2, on = ['impression_id','user_id'], how = 'left')
big_merge = pd.merge(big_merge, big_merge_cat3, on = ['impression_id','user_id'], how = 'left')
big_merge = pd.merge(big_merge, big_merge_ptype, on = ['impression_id','user_id'], how = 'left')
big_merge = pd.merge(big_merge, big_merge_price, on = ['impression_id','user_id'], how = 'left')
big_merge = pd.merge(big_merge, session_count, on = ['impression_id','user_id'], how = 'left')
big_merge.head(10)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,cat1_0,cat1_1,cat1_2,...,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,session_count
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,2.0,0.0,0.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,0.0,2.0,0.0,...,0.0,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,7.0
5,3416a75f4cea9109507cacd8e2f2aefc,2018-11-15 00:03:00,67352,409,latest,1,0,3.0,2.0,0.0,...,2.0,0.0,1.0,1.0,1.0,2.0,1.0,3.0,0.0,7.0
6,f457c545a9ded88f18ecee47145a72c0,2018-11-15 00:03:00,64356,190,intermediate,0,0,0.0,2.0,0.0,...,0.0,0.0,1.0,0.0,2.0,2.0,0.0,0.0,1.0,6.0
7,72b32a1f754ba1c09b3695e0cb6cde7f,2018-11-15 00:04:00,27329,481,latest,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
8,fc490ca45c00b1249bbe3554a4fdf6fb,2018-11-15 00:05:00,83318,386,old,0,0,0.0,2.0,0.0,...,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,8.0
9,d2ddea18f00665ce8623e36bd4e3c7c5,2018-11-15 00:05:00,70206,190,latest,0,0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,5.0


In [26]:
big_merge.describe(include = 'all')

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,cat1_0,cat1_1,cat1_2,...,price_11,price_12,price_13,price_14,price_15,price_16,price_17,price_18,price_19,session_count
count,237609,237609,237609.000000,237609.000000,237609,237609.000000,237609.000000,236022.000000,236022.000000,236022.000000,...,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000
unique,237609,36461,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,408a9c4a79800232ac656249af3162eb,2018-12-04 00:01:00,NaN,NaN,latest,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,46,NaN,NaN,129216,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,2018-11-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2018-12-13 07:43:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,46454.526828,249.099971,NaN,0.361312,0.045714,1.390777,4.276856,0.000008,...,1.783101,1.718306,1.971130,1.728140,1.900255,1.863085,1.572930,1.985641,1.796659,12.089187
std,NaN,NaN,26802.726666,135.213609,NaN,0.480382,0.208864,2.909922,7.348156,0.002911,...,3.687532,3.592234,4.256947,3.542051,3.914105,3.898724,3.429845,3.864638,3.773537,14.000254
min,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,NaN,NaN,23197.000000,163.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000


In [27]:
#ONEHOT CREATION
# dummy_app=pd.get_dummies(master['app_code'],drop_first=True)
#dummy_device=pd.get_dummies(master['device_type'],drop_first=True)
dummy_os=pd.get_dummies(big_merge['os_version'])

counts = big_merge.app_code.value_counts()
repl = counts[counts <= 5900 ].index
main_app_codes = counts[counts > 5900].index
dummy_app = pd.get_dummies(big_merge.app_code.replace(repl, 'other'),prefix = 'app')
main_app_codes = main_app_codes.tolist()

df=pd.concat([big_merge,dummy_os,dummy_app],axis=1)
df.head(5)

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click,cat1_0,cat1_1,cat1_2,...,app_32,app_127,app_129,app_190,app_207,app_296,app_371,app_385,app_386,app_other
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0,0.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
type(main_app_codes)



list

In [29]:
#DROPPING UNHANDLED VARIABLES
regdf = df.drop(['impression_id','impression_time','user_id','app_code','os_version','ptype_other','cat3_other'],axis = 1)
regdf = regdf.dropna(axis = 0, how = 'any')
regdf.head(5)
regdf.describe(include = 'all')

,is_4G,is_click,cat1_0,cat1_1,cat1_2,cat1_4,cat1_5,cat1_6,cat1_7,cat1_8,...,app_32,app_127,app_129,app_190,app_207,app_296,app_371,app_385,app_386,app_other
count,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,...,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000,236022.000000
mean,0.361259,0.045780,1.390777,4.276856,0.000008,3.392400,0.000004,0.544305,2.054635,1.915097,...,0.025277,0.045746,0.029603,0.073269,0.142059,0.031328,0.039395,0.029472,0.129166,0.417567
std,0.480366,0.209007,2.909922,7.348156,0.002911,6.423035,0.002058,1.521456,4.225658,3.882994,...,0.156966,0.208934,0.169490,0.260577,0.349111,0.174202,0.194532,0.169125,0.335384,0.493159
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,2.000000,5.000000,0.000000,4.000000,0.000000,0.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,96.000000,209.000000,1.000000,241.000000,1.000000,64.000000,166.000000,152.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
regdf.head(20)

,is_4G,is_click,cat1_0,cat1_1,cat1_2,cat1_4,cat1_5,cat1_6,cat1_7,cat1_8,...,app_32,app_127,app_129,app_190,app_207,app_296,app_371,app_385,app_386,app_other
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,1
5,1,0,3.0,2.0,0.0,5.0,0.0,1.0,0.0,3.0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0.0,2.0,0.0,2.0,0.0,0.0,4.0,0.0,...,0,0,0,1,0,0,0,0,0,0
7,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
9,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
sum(regdf['is_click'])

10805

In [32]:
regdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236022 entries, 0 to 237608
Data columns (total 89 columns):
is_4G            236022 non-null int64
is_click         236022 non-null int64
cat1_0           236022 non-null float64
cat1_1           236022 non-null float64
cat1_2           236022 non-null float64
cat1_4           236022 non-null float64
cat1_5           236022 non-null float64
cat1_6           236022 non-null float64
cat1_7           236022 non-null float64
cat1_8           236022 non-null float64
cat1_9           236022 non-null float64
cat1_10          236022 non-null float64
cat1_11          236022 non-null float64
cat1_12          236022 non-null float64
cat1_13          236022 non-null float64
cat1_14          236022 non-null float64
cat1_15          236022 non-null float64
cat1_16          236022 non-null float64
cat1_17          236022 non-null float64
cat2_3           236022 non-null float64
cat2_8           236022 non-null float64
cat2_9           236022 non-null

In [33]:
regdf.columns.tolist()

['is_4G',
 'is_click',
 'cat1_0',
 'cat1_1',
 'cat1_2',
 'cat1_4',
 'cat1_5',
 'cat1_6',
 'cat1_7',
 'cat1_8',
 'cat1_9',
 'cat1_10',
 'cat1_11',
 'cat1_12',
 'cat1_13',
 'cat1_14',
 'cat1_15',
 'cat1_16',
 'cat1_17',
 'cat2_3',
 'cat2_8',
 'cat2_9',
 'cat2_38',
 'cat2_39',
 'cat2_42',
 'cat2_44',
 'cat2_51',
 'cat2_57',
 'cat2_74',
 'cat2_other',
 'cat3_36',
 'cat3_62',
 'cat3_73',
 'cat3_84',
 'cat3_113',
 'cat3_114',
 'cat3_151',
 'cat3_157',
 'cat3_159',
 'cat3_162',
 'cat3_223',
 'cat3_244',
 'cat3_252',
 'cat3_279',
 'cat3_292',
 'ptype_2106',
 'ptype_2874',
 'ptype_5036',
 'ptype_5164',
 'ptype_5609',
 'ptype_5622',
 'ptype_7455',
 'ptype_7791',
 'ptype_10412',
 'price_0',
 'price_1',
 'price_2',
 'price_3',
 'price_4',
 'price_5',
 'price_6',
 'price_7',
 'price_8',
 'price_9',
 'price_10',
 'price_11',
 'price_12',
 'price_13',
 'price_14',
 'price_15',
 'price_16',
 'price_17',
 'price_18',
 'price_19',
 'session_count',
 'intermediate',
 'latest',
 'old',
 'app_3',
 'app_32'

In [34]:
nan_check = [regdf.isnull().sum()]
len(nan_check)
type(nan_check)
nan_check[0]

is_4G        0
is_click     0
cat1_0       0
cat1_1       0
cat1_2       0
            ..
app_296      0
app_371      0
app_385      0
app_386      0
app_other    0
Length: 89, dtype: int64

In [35]:
cat1_cols.append('item_id')
cat2_cols.append('item_id')
cat3_cols.append('item_id')
ptype_cols.append('item_id')
price_cols.append('item_id')

#prepping test data
test_merge_key = pd.merge(view[['user_id','session_id','server_time','item_id']], test, on = 'user_id', how = 'left')
test_merge_key.head(10)
#time difference calculation
test_merge_key['time_diff'] = test_merge_key['impression_time'] - test_merge_key['server_time']
test_merge_key['time_diff_s'] = test_merge_key['time_diff']/np.timedelta64(1,'s')
test_merge_key['time_diff_h'] = test_merge_key['time_diff']/np.timedelta64(1,'h')
test_merge_key['time_diff_d'] = test_merge_key['time_diff']/np.timedelta64(1,'D')

test_merge_key2 = test_merge_key[test_merge_key.time_diff_s >= 0]

test_merge_cat1 = pd.merge(test_merge_key2, itm[cat1_cols] , on = 'item_id', how = 'left')
cat1_cols.remove('item_id')
test_merge_cat1 = test_merge_cat1.groupby(['impression_id','user_id'],as_index = False)[cat1_cols].sum()

test_merge_cat2 = pd.merge(test_merge_key2, itm[cat2_cols] , on = 'item_id', how = 'left')
cat2_cols.remove('item_id')
test_merge_cat2 = test_merge_cat2.groupby(['impression_id','user_id'],as_index = False)[cat2_cols].sum()

test_merge_cat3 = pd.merge(test_merge_key2, itm[cat3_cols] , on = 'item_id', how = 'left')
cat3_cols.remove('item_id')
test_merge_cat3 = test_merge_cat3.groupby(['impression_id','user_id'],as_index = False)[cat3_cols].sum()

test_merge_ptype = pd.merge(test_merge_key2, itm[ptype_cols] , on = 'item_id', how = 'left')
ptype_cols.remove('item_id')
test_merge_ptype = test_merge_ptype.groupby(['impression_id','user_id'],as_index = False)[ptype_cols].sum()

test_merge_price = pd.merge(test_merge_key2, itm[price_cols] , on = 'item_id', how = 'left')
price_cols.remove('item_id')
test_merge_price = test_merge_price.groupby(['impression_id','user_id'],as_index = False)[price_cols].sum()

test_session_count = test_merge_key2.groupby(['user_id','impression_id'],as_index = False).agg({ 'session_id' : ['nunique']})
test_session_count.columns = ['user_id','impression_id','session_count']

# merging all variables
test_merge = pd.merge(test, test_merge_cat1 , on = ['impression_id','user_id'], how = 'left')
test_merge = pd.merge(test_merge, test_merge_cat2, on = ['impression_id','user_id'], how = 'left')
test_merge = pd.merge(test_merge, test_merge_cat3, on = ['impression_id','user_id'], how = 'left')
test_merge = pd.merge(test_merge, test_merge_ptype, on = ['impression_id','user_id'], how = 'left')
test_merge = pd.merge(test_merge, test_merge_price, on = ['impression_id','user_id'], how = 'left')
test_merge = pd.merge(test_merge, test_session_count, on = ['impression_id','user_id'], how = 'left')

#ONEHOT CREATION
test_dummy_os=pd.get_dummies(test_merge['os_version'])
test_app_codes = test_merge.app_code.unique().tolist()
test_replace = [x for x in test_app_codes if x not in main_app_codes]
test_dummy_app = pd.get_dummies(test_merge.app_code.replace(test_replace, 'other'),prefix = 'app')

test_df=pd.concat([test_merge,test_dummy_os,test_dummy_app],axis=1)
test_df.head(5)
#DROPPING UNHANDLED VARIABLES
test_regdf = test_df.drop(['impression_id','impression_time','user_id','app_code','os_version','ptype_other','cat3_other'],axis = 1)
test_regdf = test_regdf.dropna(axis = 0, how = 'any')
test_regdf.head(5)


,is_4G,cat1_0,cat1_1,cat1_2,cat1_4,cat1_5,cat1_6,cat1_7,cat1_8,cat1_9,...,app_32,app_127,app_129,app_190,app_207,app_296,app_371,app_385,app_386,app_other
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
1,0,1.0,14.0,0.0,2.0,0.0,0.0,2.0,3.0,11.0,...,0,0,0,0,0,0,0,0,0,1
2,1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,...,0,0,0,0,0,1,0,0,0,0
3,1,1.0,8.0,0.0,0.0,0.0,8.0,0.0,3.0,10.0,...,0,0,0,0,1,0,0,0,0,0
4,1,2.0,5.0,0.0,2.0,0.0,1.0,2.0,2.0,1.0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
main_app_codes.tolist()


AttributeError: 'list' object has no attribute 'tolist'

In [37]:
test_regdf.columns.tolist()


['is_4G',
 'cat1_0',
 'cat1_1',
 'cat1_2',
 'cat1_4',
 'cat1_5',
 'cat1_6',
 'cat1_7',
 'cat1_8',
 'cat1_9',
 'cat1_10',
 'cat1_11',
 'cat1_12',
 'cat1_13',
 'cat1_14',
 'cat1_15',
 'cat1_16',
 'cat1_17',
 'cat2_3',
 'cat2_8',
 'cat2_9',
 'cat2_38',
 'cat2_39',
 'cat2_42',
 'cat2_44',
 'cat2_51',
 'cat2_57',
 'cat2_74',
 'cat2_other',
 'cat3_36',
 'cat3_62',
 'cat3_73',
 'cat3_84',
 'cat3_113',
 'cat3_114',
 'cat3_151',
 'cat3_157',
 'cat3_159',
 'cat3_162',
 'cat3_223',
 'cat3_244',
 'cat3_252',
 'cat3_279',
 'cat3_292',
 'ptype_2106',
 'ptype_2874',
 'ptype_5036',
 'ptype_5164',
 'ptype_5609',
 'ptype_5622',
 'ptype_7455',
 'ptype_7791',
 'ptype_10412',
 'price_0',
 'price_1',
 'price_2',
 'price_3',
 'price_4',
 'price_5',
 'price_6',
 'price_7',
 'price_8',
 'price_9',
 'price_10',
 'price_11',
 'price_12',
 'price_13',
 'price_14',
 'price_15',
 'price_16',
 'price_17',
 'price_18',
 'price_19',
 'session_count',
 'intermediate',
 'latest',
 'old',
 'app_3',
 'app_32',
 'app_127',

In [38]:
test_regdf.describe(include = 'all')

,is_4G,cat1_0,cat1_1,cat1_2,cat1_4,cat1_5,cat1_6,cat1_7,cat1_8,cat1_9,...,app_32,app_127,app_129,app_190,app_207,app_296,app_371,app_385,app_386,app_other
count,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000,90675.0,90675.000000,90675.000000,90675.000000,90675.000000,...,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000,90675.000000
mean,0.357872,1.635732,5.088051,0.000022,4.004444,0.0,0.608018,2.382035,2.243562,5.607191,...,0.019101,0.035379,0.025884,0.074872,0.158941,0.035081,0.029975,0.025542,0.157629,0.406077
std,0.479377,3.274513,8.341218,0.004696,7.089636,0.0,1.630876,4.694217,4.308109,9.363181,...,0.136881,0.184737,0.158789,0.263186,0.365623,0.183986,0.170520,0.157765,0.364395,0.491102
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,2.000000,0.000000,2.000000,0.0,0.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,6.000000,0.000000,5.000000,0.0,1.000000,3.000000,3.000000,7.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,58.000000,173.000000,1.000000,241.000000,0.0,36.000000,92.000000,152.000000,198.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [39]:
nan_check = [test_regdf.isnull().sum()]
len(nan_check)
sum(nan_check[0])

0

In [40]:
#TRAIN TEST SPLIT

#using SMOTE
from sklearn.model_selection import train_test_split
X=regdf.drop(['is_click'],axis=1)
y=regdf['is_click']
#from imblearn.under_sampling import RandomUnderSampler
#rus = RandomUnderSampler(random_state=0)
#X_r, y_r = rus.fit_resample(X, y)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_r, y_r = sm.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_r, y_r, test_size=0.3, random_state=42)

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [41]:
X_r.head(5)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [42]:
# #using undersampling
# from sklearn.model_selection import train_test_split
# X=regdf.drop(['is_click'],axis=1)
# y=regdf['is_click']
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# X_r1, y_r1 = rus.fit_resample(X, y)
# X_train, X_test, y_train, y_test = train_test_split(X_r1, y_r1, test_size=0.33, random_state=42)

In [43]:
#logistic regression fitting
#dict1 = {0:.08, 1:.92}
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='newton-cg',class_weight = 'balanced' ,multi_class='ovr',max_iter = 100).fit(X_r, y_r)
#clf.predict(X_train,y_train)
pred = clf.predict(X_test) 
clf.score(X_test, y_test)

0.6300404792386647

In [44]:
X.columns.tolist()


['is_4G',
 'cat1_0',
 'cat1_1',
 'cat1_2',
 'cat1_4',
 'cat1_5',
 'cat1_6',
 'cat1_7',
 'cat1_8',
 'cat1_9',
 'cat1_10',
 'cat1_11',
 'cat1_12',
 'cat1_13',
 'cat1_14',
 'cat1_15',
 'cat1_16',
 'cat1_17',
 'cat2_3',
 'cat2_8',
 'cat2_9',
 'cat2_38',
 'cat2_39',
 'cat2_42',
 'cat2_44',
 'cat2_51',
 'cat2_57',
 'cat2_74',
 'cat2_other',
 'cat3_36',
 'cat3_62',
 'cat3_73',
 'cat3_84',
 'cat3_113',
 'cat3_114',
 'cat3_151',
 'cat3_157',
 'cat3_159',
 'cat3_162',
 'cat3_223',
 'cat3_244',
 'cat3_252',
 'cat3_279',
 'cat3_292',
 'ptype_2106',
 'ptype_2874',
 'ptype_5036',
 'ptype_5164',
 'ptype_5609',
 'ptype_5622',
 'ptype_7455',
 'ptype_7791',
 'ptype_10412',
 'price_0',
 'price_1',
 'price_2',
 'price_3',
 'price_4',
 'price_5',
 'price_6',
 'price_7',
 'price_8',
 'price_9',
 'price_10',
 'price_11',
 'price_12',
 'price_13',
 'price_14',
 'price_15',
 'price_16',
 'price_17',
 'price_18',
 'price_19',
 'session_count',
 'intermediate',
 'latest',
 'old',
 'app_3',
 'app_32',
 'app_127',

In [45]:
test_regdf.columns.tolist()

['is_4G',
 'cat1_0',
 'cat1_1',
 'cat1_2',
 'cat1_4',
 'cat1_5',
 'cat1_6',
 'cat1_7',
 'cat1_8',
 'cat1_9',
 'cat1_10',
 'cat1_11',
 'cat1_12',
 'cat1_13',
 'cat1_14',
 'cat1_15',
 'cat1_16',
 'cat1_17',
 'cat2_3',
 'cat2_8',
 'cat2_9',
 'cat2_38',
 'cat2_39',
 'cat2_42',
 'cat2_44',
 'cat2_51',
 'cat2_57',
 'cat2_74',
 'cat2_other',
 'cat3_36',
 'cat3_62',
 'cat3_73',
 'cat3_84',
 'cat3_113',
 'cat3_114',
 'cat3_151',
 'cat3_157',
 'cat3_159',
 'cat3_162',
 'cat3_223',
 'cat3_244',
 'cat3_252',
 'cat3_279',
 'cat3_292',
 'ptype_2106',
 'ptype_2874',
 'ptype_5036',
 'ptype_5164',
 'ptype_5609',
 'ptype_5622',
 'ptype_7455',
 'ptype_7791',
 'ptype_10412',
 'price_0',
 'price_1',
 'price_2',
 'price_3',
 'price_4',
 'price_5',
 'price_6',
 'price_7',
 'price_8',
 'price_9',
 'price_10',
 'price_11',
 'price_12',
 'price_13',
 'price_14',
 'price_15',
 'price_16',
 'price_17',
 'price_18',
 'price_19',
 'session_count',
 'intermediate',
 'latest',
 'old',
 'app_3',
 'app_32',
 'app_127',

In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.69      0.47      0.56     67466
           1       0.60      0.79      0.68     67665

    accuracy                           0.63    135131
   macro avg       0.65      0.63      0.62    135131
weighted avg       0.65      0.63      0.62    135131



In [47]:
# from sklearn.svm import SVC
# svm = SVC(gamma='auto')
# svm .fit(X_train, y_train) 
# pred1 = svm.predict(X_test) 
# svm.score(X_test, y_test)

In [48]:
# from sklearn.neural_network import MLPClassifier
# nn = MLPClassifier(hidden_layer_sizes=(100,40,10 ), activation='relu', alpha=0.0001, max_iter=4000)
# nn .fit(X_train, y_train) 
# pred1 = nn.predict(X_test) 
# nn.score(X_test, y_test)

In [49]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))
    
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [50]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
scores = cross_val_score(xgb_model, X_train, y_train, scoring="neg_mean_squared_error", cv=5)
display_scores(np.sqrt(-scores))


Scores: [0.30156632 0.30366479 0.30064947 0.30272575 0.30070221]
Mean: 0.302
Std: 0.001


In [51]:
xgb_model.fit(X_train,y_train)
pred_xgb = xgb_model.predict(X_test)
roc_auc_score(y_test, pred_xgb)

NameError: name 'roc_auc_score' is not defined

In [52]:
pred_xgb2 = xgb_model.predict_proba(X_test)
sum(pred_xgb2)

array([67606.1 , 67524.88], dtype=float32)

In [53]:
sum(pred_xgb)


56168

In [54]:
#AREA UNDER CURVE
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, pred)

0.629799388440556

In [55]:
test_pred = clf.predict_proba(test_regdf)[:,1]
sum(test_pred)

39563.95008716616

In [56]:
test_pred[10]


0.5826919379035973

In [57]:
xgb_model.fit(X_r,y_r)
test_regdf2 = test_regdf.as_matrix()
test_pred_xgb = xgb_model.predict_proba(test_regdf2)[:,1]
sum(test_pred_xgb)
roc_auc_score(y_test, test_pred_xgb)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


ValueError: Found input variables with inconsistent numbers of samples: [135131, 90675]

In [58]:
test_pred_xgb[5]


0.046005327

In [59]:
submit_data = pd.DataFrame()
submit_data['impression_id'] = test_df['impression_id']
submit_data['is_click'] = test_pred_xgb
submit_data.head(5)
submit_data.describe()

,is_click
count,90675.000000
mean,0.183436
std,0.122837
min,0.009058
25%,0.049377
50%,0.191158
75%,0.268378
max,0.733641


In [60]:
submit_data.to_csv('submit2_xgb.csv',index = False)